<a href="https://colab.research.google.com/github/sophiyageorge/Fine-Turning-Model/blob/main/distil_bert_lora_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from datasets import Dataset

In [2]:
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification

In [3]:
from peft import LoraConfig,get_peft_model

In [4]:
import pandas as pd

In [23]:
dataset = pd.read_csv('cleaned_lora.csv')

In [25]:
dataset.shape

(10000, 3)

In [ ]:
dataset = df.rename(columns={'useful':'label'})

In [ ]:
dataset = dataset[['text','label']]

In [ ]:
dataset.loc[:,'label'] = dataset['label'].apply(lambda x:1 if x>=3 else 0)

In [ ]:
dataset.to_csv('cleaned_lora.csv')

In [8]:
dataset = Dataset.from_pandas(dataset)

In [9]:
dataset = dataset.train_test_split(test_size=0.2)

In [10]:
model_name = 'distilbert-base-uncased'

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [12]:
def tokenize(batch):
    return tokenizer(batch['text'],truncation=True, padding='max_length')

dataset = dataset.map(tokenize,batched = True)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [13]:
# loading base model
model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels =2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# lora config
lora_config = LoraConfig(
    task_type="SEQ_CLS",
    r=8,
    lora_alpha = 16,
    target_modules=['q_lin','v_lin'], # works for DistilBERT
    lora_dropout=0.1
)

In [16]:
model = get_peft_model(model,lora_config)

In [17]:
# training
training_args = TrainingArguments(output_dir='./distilbert-lora-output',
                                  per_device_train_batch_size = 2,
                                  learning_rate=2e-5,
                                  num_train_epochs=1,
                                  eval_strategy='epoch'
)

In [20]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [18]:
trainer = Trainer(model = model,args = training_args,train_dataset=dataset['train'], eval_dataset=dataset['test'],processing_class=tokenizer)

In [21]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
model.save_pretrained('./distilbert-lora')

# evaluating

In [ ]:
from peft import PeftModel

In [ ]:
base_model_name = 'distilbert-base-uncased'
lora_model_path = './distilbert-lora'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
base_model = AutoModelForSequenceClassification.from_pretrained(lora_model_name)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = PeftModel.from_pretrained(base_model,lora_model_path)

C:\Users\mathe\AppData\Roaming\Python\Python312\site-packages\peft\tuners\tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [ ]:
model.eval()

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): DistilBertSdpaAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=768, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=76

# Making a prediction

In [ ]:
import torch

In [ ]:
text = 'the food is good!'

In [ ]:
inputs = tokenizer(text,return_tensors='pt', padding = 'max_length',max_length=128,truncation=True)

In [ ]:
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits,dim =-1)

In [ ]:
predictions.item()

0